### Generate Training Data

embed watermark for this image

then save it, its filename represents what embedded

In [2]:
import os
import time

import numpy as np
import cv2 as cv
import pandas as pd
from importlib import reload
import torch as tc
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Compose, RandomResizedCrop
from torch.utils.data import Dataset
from PIL import Image
from matplotlib import pyplot as plt
import models.context
import models.transforms
reload(models.context);reload(models.transforms)
from models.context import WatermarkContext
from models.transforms import (
    CropToBlocks, FromBlocks, DiscreteWaveletTransform, InverseDiscreteWaveletTransform, EmbedBitUsingChannelReference,
    CollectStatistic, DiscreteCosineTransform, InverseDiscreteCosineTransform, ExtractBitUsingChannelReference,
    JPEGCompression, ToSignalValueRange, FromSignalValueRange,
)

class OriginalImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        file_list = []
        for file in os.listdir(img_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                file_list.append(os.path.join(img_dir, file))
        assert len(file_list) > 0, 'img_dir:{} has no images'.format(img_dir)
        self.img_dir = img_dir
        self.file_list = file_list
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx])
        # image = read_image(self.file_list[idx])
        if self.transform:
            image = self.transform(image)
        return image

ctx = WatermarkContext(wm_strength=2)
error_block = None
embeder = Compose([
    ToTensor(),
    ToSignalValueRange(ctx),
    CropToBlocks(ctx),
    DiscreteWaveletTransform(ctx),
    DiscreteCosineTransform(ctx),
    EmbedBitUsingChannelReference(ctx, print_error_block=error_block),
    InverseDiscreteCosineTransform(ctx),
    InverseDiscreteWaveletTransform(ctx),
    FromBlocks(ctx),
    CollectStatistic(ctx),
    FromSignalValueRange(ctx),
])

attacker = Compose([
    JPEGCompression(ctx, jpeg_quality=80, to_file='../data/dst1.jpg'),
])

extractor = Compose([
    ToSignalValueRange(ctx),
    CropToBlocks(ctx),
    DiscreteWaveletTransform(ctx),
    DiscreteCosineTransform(ctx),
    ExtractBitUsingChannelReference(ctx, print_error_block=error_block),
])


dataset = OriginalImageDataset('/Users/jiahao.chen/Downloads/wm-nowm/valid/no-watermark')

import time
BER, PSNR = 0, tc.zeros(3)
for i, img in enumerate(dataset):
    start_time = time.time()
    dst = embeder(img)
    print('elapsed:', time.time() - start_time)
    dst[dst > 1] = 1
    dst[dst < 0] = 0
    dst = attacker(dst)
    wm = extractor(dst)
    ber, psnr = (ctx.errors['BER'], ctx.errors['PSNR'])
    BER += ber
    PSNR += psnr
    if i % 20 == 0:
        print(BER/(i+1), PSNR/(i+1))

    # test
    print(ctx.errors)

    break

elapsed: 0.011002779006958008
0.16176470588235295 tensor([    inf,     inf, 29.2865])
{'EmbedElapsed': 0.00025773048400878906, 'PSNR': tensor([    inf,     inf, 29.2865]), 'overflow': [0.0, 0.0, 2.8529411764705883], 'underflow': [0.022058823529411766, 0.022058823529411766, 3.7426470588235294], 'BER': 0.16176470588235295}


In [184]:
ctx = WatermarkContext(wm_strength=3, )
error_block = None
embeder = Compose([
    ToTensor(),
    ToSignalValueRange(ctx),
    CropToBlocks(ctx),
    DiscreteWaveletTransform(ctx),
    DiscreteCosineTransform(ctx),
    EmbedBitUsingChannelReference(ctx, print_error_block=error_block, use_numpy=False),
    InverseDiscreteCosineTransform(ctx),
    InverseDiscreteWaveletTransform(ctx),
    FromBlocks(ctx),
    FromSignalValueRange(ctx),
    CollectStatistic(ctx),
])

attacker = Compose([
    JPEGCompression(ctx, jpeg_quality=90, to_file='../data/dst1.jpg'),
])

extractor = Compose([
    ToSignalValueRange(ctx),
    CropToBlocks(ctx),
    DiscreteWaveletTransform(ctx),
    DiscreteCosineTransform(ctx),
    ExtractBitUsingChannelReference(ctx, print_error_block=error_block),
])

src = Image.open('../data/flower.jpg')
with tc.no_grad():
    st=time.time()
    dst = embeder(src)
    print('emb cost', time.time()-st);
    dst = attacker(dst)
    st=time.time()
    wm = extractor(dst)
    print('ext cost', time.time()-st)
print(ctx.errors)

emb cost 1.1652836799621582
ext cost 0.45702672004699707
{'EmbedElapsed': 0.23291587829589844, 'PSNR': tensor([9.1850, 8.4378, 6.9023]), 'overflow': [347.81839984650804, 349.61809286262474, 139.10830775134306], 'underflow': [0.0, 0.0, 0.0], 'BER': 0.00028779739063699157}


In [3]:
# single object detection
import time
from torchvision.transforms import RandomRotation, InterpolationMode
import models.context
import models.transforms
reload(models.context);reload(models.transforms)
from models.context import WatermarkContext
from models.transforms import (
    CropToBlocks, FromBlocks, DiscreteWaveletTransform, InverseDiscreteWaveletTransform, EmbedBitUsingChannelReference,
    CollectStatistic, DiscreteCosineTransform, InverseDiscreteCosineTransform, ExtractBitUsingChannelReference,
    JPEGCompression, ToSignalValueRange, FromSignalValueRange, AdaptiveRegionSelectionWatermark,
)

ctx = WatermarkContext(wm_strength=2)
embeder = Compose([
    AdaptiveRegionSelectionWatermark(ctx,
                                     input_format=AdaptiveRegionSelectionWatermark.INPUT_FORMAT_PIL,
                                     best_performance=False,
                                     inplace=False),
])
attacker = Compose([
    JPEGCompression(ctx, jpeg_quality=60, to_file='../data/dst1.jpg'),
    # RandomRotation(4, interpolation=InterpolationMode.BILINEAR),
])
extractor = Compose([
    AdaptiveRegionSelectionWatermark(ctx, is_embed=False,
                                     input_format=AdaptiveRegionSelectionWatermark.INPUT_FORMAT_TENSOR,
                                     best_performance=False),
])

# define
dataset = OriginalImageDataset('/Users/jiahao.chen/Downloads/wm-nowm/valid/no-watermark')
img = np.array(dataset[802])

BER, PSNR, Confidence = 0, tc.zeros(3), 0.
times = np.array([0, 0, 0])
src = None
for i, img in enumerate(dataset):
    bit = bool(np.random.randint(0, 2, 1)[0] == 1)
    ctx.wm_bits = bit
    start_time = time.time()
    dst = embeder(img)
    times[0] += time.time() - start_time; start_time = time.time()
    dst = (dst/255).transpose(2,1,0)
    dst = attacker(dst)
    times[1] += time.time() - start_time; start_time = time.time()
    try:
        wm = extractor(dst)
    except AssertionError as e:
        print(i, e)
        img.save('../data/dst2.jpg')
        src = dst
        continue
    times[2] += time.time() - start_time; start_time = time.time()
    c, psnr, ber = ctx.errors.get('Confidence'), ctx.errors['PSNR'], 0
    Confidence += abs(c)
    PSNR += psnr
    BER += (c > 0) != bit
    if i % 20 == 0:
        print('BER:', BER/(i+1),
              '\tPSNR:', (PSNR/(i+1)).numpy(),
              '\tConfidence:', int(Confidence/(i+1)*1000),
              ((times*1000)/(i+1)).astype(int))

    # # test
    # print(ctx.errors)

    # break


BER: 0.0 	PSNR: [66.71425883 66.30732545 44.67409201] 	Confidence: 22 [0 0 0]
BER: 0.09523809523809523 	PSNR: [68.28947186 68.10085787 46.58370883] 	Confidence: 28 [0 0 0]
BER: 0.0975609756097561 	PSNR: [68.15694436 68.22821593 46.25048108] 	Confidence: 26 [0 0 0]
BER: 0.11475409836065574 	PSNR: [67.94312573 67.96233464 46.0069513 ] 	Confidence: 25 [0 0 0]
BER: 0.09876543209876543 	PSNR: [67.71730824 67.7012068  45.75570987] 	Confidence: 23 [0 0 0]
BER: 0.0891089108910891 	PSNR: [67.57077626 67.60846285 45.633969  ] 	Confidence: 23 [0 0 0]
BER: 0.08264462809917356 	PSNR: [67.70083471 67.7960121  45.78295003] 	Confidence: 23 [0 0 0]
BER: 0.0851063829787234 	PSNR: [67.68353784 67.77837153 45.77625828] 	Confidence: 24 [0 0 0]
BER: 0.11180124223602485 	PSNR: [67.67808384 67.78317788 45.76622138] 	Confidence: 24 [0 0 0]
BER: 0.10497237569060773 	PSNR: [67.66614719 67.78851415 45.78248724] 	Confidence: 24 [0 0 0]
BER: 0.10945273631840796 	PSNR: [67.67954641 67.80474996 45.84215948] 	Confiden

KeyboardInterrupt: 

In [76]:
# single object detection
import time
from torchvision.transforms import RandomRotation, InterpolationMode
import models.context
import models.transforms
import utils
reload(models.context);reload(models.transforms)
reload(utils)
from models.context import WatermarkContext
from models.transforms import (
    CropToBlocks, FromBlocks, DiscreteWaveletTransform, InverseDiscreteWaveletTransform, EmbedBitUsingChannelReference,
    CollectStatistic, DiscreteCosineTransform, InverseDiscreteCosineTransform, ExtractBitUsingChannelReference,
    JPEGCompression, ToSignalValueRange, FromSignalValueRange, AdaptiveRegionSelectionWatermark,
)

ctx = WatermarkContext(wm_strength=4, wm_bits=True)
embeder = AdaptiveRegionSelectionWatermark(ctx,
                                     input_format=AdaptiveRegionSelectionWatermark.INPUT_FORMAT_PIL,
                                     best_performance=False, inplace=False, save_kp_img_to='../data/kp1.jpg')
attacker = Compose([
    JPEGCompression(ctx, jpeg_quality=35, to_file='../data/dst1.jpg'),
    # RandomRotation(1, interpolation=InterpolationMode.BILINEAR),
])
extractor = AdaptiveRegionSelectionWatermark(ctx, is_embed=False,
                                     input_format=AdaptiveRegionSelectionWatermark.INPUT_FORMAT_TENSOR,
                                     best_performance=False, save_kp_img_to='../data/kp2.jpg')

src = Image.open('../data/flower.jpg')
with tc.no_grad():
    dst = embeder(src)
    dst = (dst/255).transpose(2, 0, 1)
    dst = attacker(dst)
    # im = Image.fromarray((dst*255).numpy().astype(np.uint8).transpose(1,2,0));
    # im.save('jfio34.jpg')
    wm = extractor(dst)
print(wm, ctx.errors)

[169.38984088 169.45947463  37.49789015] psnr
0.006957287 {'ORBElapsed': 0.07048606872558594, 'PSNR': array([169.38984088, 169.45947463,  37.49789015]), 'TotalElapsed': 0.13501286506652832, 'Confidence': 0.006957287}
